In [2]:
import json
import re
import pandas as pandas
from slugify.slugify import slugify

from nibe_modbus.heatpump import Model

model = Model.F1255

file = model.get_data_file(extension='csv')

In [3]:
data = pandas.read_csv(file, sep=";", skiprows=4, encoding='latin1', index_col=False, skipinitialspace=True)

# lowercase column names
data.columns = map(str.lower, data.columns)
data.set_index('id', inplace=True)
data


,title,info,unit,size,factor,min,max,default,mode
id,,,,,,,,,
32260,NIBE Inverter 216-state,NaN,NaN,u8,1,0,0,0,R
40004,BT1 Outdoor Temperature,Current outdoor temperature,°C,s16,10,0,0,0,R
40005,EP23-BT2 Supply temp S4,Supply temperature for system 4,°C,s16,10,0,0,0,R
40006,EP22-BT2 Supply temp S3,Supply temperature for system 3,°C,s16,10,0,0,0,R
40007,EP21-BT2 Supply temp S2,Supply temperature for system 2,°C,s16,10,0,0,0,R
...,...,...,...,...,...,...,...,...,...
49378,External ERS 2 accessory bypass set temp.,NaN,°C,u8,1,5,30,25,R/W
49379,External ERS 4 accessory bypass at heat,NaN,NaN,u8,1,0,1,0,R/W
49380,External ERS 3 accessory bypass at heat,NaN,NaN,u8,1,0,1,0,R/W


In [4]:
data['unit'] = data['unit'].replace(r'^\s*$', pandas.NA, regex=True)

# Make name
data['name'] = pandas.Series(data.index, index=data.index).combine(data['title'], lambda id_, title: slugify(f"{title}-{id_}")).astype('string')

# Set types
data['title'] = data['title'].astype('string')
data['info'] = data['info'].astype('string')
data['unit'] = data['unit'].str.strip().astype('string')
data['mode'] = data['mode'].str.strip().astype('string')
data['size'] = data['size'].astype('string')

data['write'] = data['mode'].map(lambda x: x == 'R/W')
valid_min_max = data['min'] != data['max']

data['min'] = data['min'].where(valid_min_max)
data['max'] = data['max'].where(valid_min_max)
data['default'] = data['default'].where(valid_min_max)
data

,title,info,unit,size,factor,min,max,default,mode,name,write
id,,,,,,,,,,,
32260,NIBE Inverter 216-state,<NA>,<NA>,u8,1,NaN,NaN,NaN,R,nibe-inverter-216-state-32260,False
40004,BT1 Outdoor Temperature,Current outdoor temperature,°C,s16,10,NaN,NaN,NaN,R,bt1-outdoor-temperature-40004,False
40005,EP23-BT2 Supply temp S4,Supply temperature for system 4,°C,s16,10,NaN,NaN,NaN,R,ep23-bt2-supply-temp-s4-40005,False
40006,EP22-BT2 Supply temp S3,Supply temperature for system 3,°C,s16,10,NaN,NaN,NaN,R,ep22-bt2-supply-temp-s3-40006,False
40007,EP21-BT2 Supply temp S2,Supply temperature for system 2,°C,s16,10,NaN,NaN,NaN,R,ep21-bt2-supply-temp-s2-40007,False
...,...,...,...,...,...,...,...,...,...,...,...
49378,External ERS 2 accessory bypass set temp.,<NA>,°C,u8,1,5.0,30.0,25.0,R/W,external-ers-2-accessory-bypass-set-temp-49378,True
49379,External ERS 4 accessory bypass at heat,<NA>,<NA>,u8,1,0.0,1.0,0.0,R/W,external-ers-4-accessory-bypass-at-heat-49379,True
49380,External ERS 3 accessory bypass at heat,<NA>,<NA>,u8,1,0.0,1.0,0.0,R/W,external-ers-3-accessory-bypass-at-heat-49380,True


In [5]:
with_equal = data['info'].where(data['info'].str.contains('=')).dropna()

In [6]:
# Mappings
re_mapping = re.compile(r'(?P<value>\d+|I)\s*=\s*(?P<key>(?:[\w +.-]+[\w]\b[+]?(?! *=)))', re.IGNORECASE)
mappings = data['info'].where(~data['info'].str.contains("encoded")).str.extractall(re_mapping)
mappings['value'] = mappings['value'].str.replace('I', '1').astype('int')
mappings = mappings.reset_index("match", drop=True)
# mappings['id'] = mappings.index
mappings = mappings.drop_duplicates()
# del mappings['id']
mappings

,value,key
id,,
40912,10,off
40912,40,active wait
40912,50,active
40913,10,shunt off
40913,20,shunt open
...,...,...
49193,0,manual
49193,1,auto controlled
49193,2,fixed delta controlled


In [7]:
mappings

,value,key
id,,
40912,10,off
40912,40,active wait
40912,50,active
40913,10,shunt off
40913,20,shunt open
...,...,...
49193,0,manual
49193,1,auto controlled
49193,2,fixed delta controlled


In [8]:
data['mappings'] = pandas.Series({k: dict(g.values) for k,g in mappings.groupby('value', level=0)})

In [9]:
data

,title,info,unit,size,factor,min,max,default,mode,name,write,mappings
id,,,,,,,,,,,,
32260,NIBE Inverter 216-state,<NA>,<NA>,u8,1,NaN,NaN,NaN,R,nibe-inverter-216-state-32260,False,NaN
40004,BT1 Outdoor Temperature,Current outdoor temperature,°C,s16,10,NaN,NaN,NaN,R,bt1-outdoor-temperature-40004,False,NaN
40005,EP23-BT2 Supply temp S4,Supply temperature for system 4,°C,s16,10,NaN,NaN,NaN,R,ep23-bt2-supply-temp-s4-40005,False,NaN
40006,EP22-BT2 Supply temp S3,Supply temperature for system 3,°C,s16,10,NaN,NaN,NaN,R,ep22-bt2-supply-temp-s3-40006,False,NaN
40007,EP21-BT2 Supply temp S2,Supply temperature for system 2,°C,s16,10,NaN,NaN,NaN,R,ep21-bt2-supply-temp-s2-40007,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
49378,External ERS 2 accessory bypass set temp.,<NA>,°C,u8,1,5.0,30.0,25.0,R/W,external-ers-2-accessory-bypass-set-temp-49378,True,NaN
49379,External ERS 4 accessory bypass at heat,<NA>,<NA>,u8,1,0.0,1.0,0.0,R/W,external-ers-4-accessory-bypass-at-heat-49379,True,NaN
49380,External ERS 3 accessory bypass at heat,<NA>,<NA>,u8,1,0.0,1.0,0.0,R/W,external-ers-3-accessory-bypass-at-heat-49380,True,NaN


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 976 entries, 32260 to 49430
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   title     976 non-null    string 
 1   info      641 non-null    string 
 2   unit      524 non-null    string 
 3   size      976 non-null    string 
 4   factor    976 non-null    int64  
 5   min       537 non-null    float64
 6   max       537 non-null    float64
 7   default   537 non-null    float64
 8   mode      976 non-null    string 
 9   name      976 non-null    string 
 10  write     976 non-null    bool   
 11  mappings  35 non-null     object 
dtypes: bool(1), float64(3), int64(1), object(1), string(6)
memory usage: 132.5+ KB


In [14]:
data['size'].unique()

<StringArray>
['u8', 's16', 'u32', 'u16', 's32', 's8']
Length: 6, dtype: string

In [12]:
del(data['mode'])
data['write'] = data['write'].map(lambda x: x or pandas.NA)


In [13]:
js = json.dumps({index: row.dropna().to_dict() for index,row in data.iterrows()}, indent=2)

print(js)

{
  "32260": {
    "title": "NIBE Inverter 216-state",
    "size": "u8",
    "factor": 1,
    "name": "nibe-inverter-216-state-32260"
  },
  "40004": {
    "title": "BT1 Outdoor Temperature",
    "info": "Current outdoor temperature",
    "unit": "\u00b0C",
    "size": "s16",
    "factor": 10,
    "name": "bt1-outdoor-temperature-40004"
  },
  "40005": {
    "title": "EP23-BT2 Supply temp S4",
    "info": "Supply temperature for system 4",
    "unit": "\u00b0C",
    "size": "s16",
    "factor": 10,
    "name": "ep23-bt2-supply-temp-s4-40005"
  },
  "40006": {
    "title": "EP22-BT2 Supply temp S3",
    "info": "Supply temperature for system 3",
    "unit": "\u00b0C",
    "size": "s16",
    "factor": 10,
    "name": "ep22-bt2-supply-temp-s3-40006"
  },
  "40007": {
    "title": "EP21-BT2 Supply temp S2",
    "info": "Supply temperature for system 2",
    "unit": "\u00b0C",
    "size": "s16",
    "factor": 10,
    "name": "ep21-bt2-supply-temp-s2-40007"
  },
  "40008": {
    "title": "BT